In [1]:
import os
import pathlib
import imageio
import numpy as np
import pandas as pd

# %matplotlib inline
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [3]:
classes = pd.read_csv('classes.csv')

In [24]:
classes

,filename,foreground,background
0,d32ea6d318626ca14a967d0c1ad3218aebfe636624a8d1...,white,black
1,4596961c789d3b41916492918797724fe75128239fefc5...,white,black
2,b6edad733399c83c8eb7a59c0d37b54e10cc0d59894e39...,white,black
3,831218e6a1a54b23d4be56c5799854e7eb978811b89215...,white,black
4,b1e3aeb0c56261c17eb71c747d116057b8da7e8c8a6845...,white,black
5,4a424e0cb845cf6fd4d9fe62875552c7b89a4e0276cf16...,white,black
6,43cf6b2ec0b0745ac2b87b4d8780f62e9050d3f5d50a1f...,white,black
7,f487cc82271cf84b4414552aa8b0a9d82d902451ebe8e8...,white,black
8,bf7691b0a79811fa068b7408cbce636a73f01ef9e971a9...,white,black
9,df33b11184427e05c8a450f921586685975fe975f57315...,white,black


In [21]:
# Glob the training data and load a single image path
training_paths = pathlib.Path('data/stage1_train').glob('*/images/*.png')
training_sorted = sorted([x for x in training_paths])
image_to_segmentation = {}
image_keys = []
for p in training_sorted:
    image_key = str(p).split('/')[-1].strip('.png')
    image_keys.append(image_key)
    seg_paths = pathlib.Path('stage1_train/' + image_key).glob('masks/*.png')
    seg_sorted = sorted([x for x in seg_paths])
    image_to_segmentation[image_key] = []
    for p_seg in seg_sorted:
        image_to_segmentation[image_key].append(str(p_seg).split('/')[-1].strip('.png'))

print('the total number of training image is: ', len(image_to_segmentation))

the total number of training image is:  670


In [30]:
color_imgs = []
for i, img_key in enumerate(classes['filename']):
    if img_key.strip('.png') in image_keys and classes['foreground'][i] == 'purple':
        color_imgs.append(img_key.strip('.png'))
        print(img_key)
print(len(color_imgs))

ed5be4b63e9506ad64660dd92a098ffcc0325195298c13c815a73773f1efc279.png
94519eb45cbe1573252623b7ea06a8b43c19c930f5c9b685edb639d0db719ab0.png
79fe419488ba98494e3baa35c6fef9662eda1efe325d0ab0ac002f5383245d96.png
353ab00e964f71aa720385223a9078b770b7e3efaf5be0f66e670981f68fe606.png
f4b7c24baf69b8752c49d0eb5db4b7b5e1524945d48e54925bff401d5658045d.png
1631352dbafb8a90f11219fffd3bea368a30bc3bad3bbe0e84e19bd720df4945.png
2c61fdcb36fd1b2944895af6204279e9f6c164ba894198b40c8b7a3c9bf500ea.png
a0325cb7aa59e9c0a75e64ba26855d8032c46161aa4bca0c01bac5e4a836485e.png
420f43d21dbaba42bf8c0995b3a2c85537876d594433770c6c6f3d6b779ec15f.png
ef3ef194e5657fda708ecbd3eb6530286ed2ba23c88efb9f1715298975c73548.png
bb61fc17daf8bdd4e16fdcf50137a8d7762bec486ede9249d92e511fcb693676.png
0e4c2e2780de7ec4312f0efcd86b07c3738d21df30bb4643659962b4da5505a3.png
1ec74a26e772966df764e063f1391109a60d803cff9d15680093641ed691bf72.png
aa58ba4512955771b4f9b459cb4e6a8adb71d11cd6cae662ec2df31d688a5fe0.png
15039b3acccc4257a1a442646a89b6e596

In [31]:
with open('purple_108', 'a') as f:
    for key in color_imgs:
        print('stage1_train/' + key, file=f)
